In [110]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np
from numpy import linalg as LA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
stop = stopwords.words('english')
stop_words = set(stopwords.words("english"))
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import time

# 1.1 Loading in Data

In [111]:
#1.1- LOAD THE DATA
texts = pd.read_csv("D:\\OneDrive\\University of Washington Seattle\\Year 4 Quarter 2 (Winter)\\INFO 371\\Problem Set\\PS4\\texts.csv", sep='\t')
texts.chunk = texts['chunk'].str.lower()

# 1.2 Inspecting Data

In [39]:
#1.2- Inspect some of the texts. Note that chunkid 1 corresponds to the first page of the text.
print('Shape of file:', texts.shape)
print("")
print('Description of file: \n', texts.dtypes.value_counts())
print("")
print("Head of file: \n", texts.head())

Shape of file: (12924, 5)

Description of file: 
 int64     3
object    2
dtype: int64

Head of file: 
                               name    size  lines  chunkid  \
0  balbulus-early-life-charlemagne  259062   4394        1   
1  balbulus-early-life-charlemagne  259062   4394        2   
2  balbulus-early-life-charlemagne  259062   4394        3   
3  balbulus-early-life-charlemagne  259062   4394        4   
4  balbulus-early-life-charlemagne  259062   4394        5   

                                               chunk  
0  \ntitle: early lives of charlemagne by eginhar...  
1  \n\nthe notes, keyed to line numbers in the so...  
2  \n         from a bronze statuette in the musé...  
3  \n                _a lui finit la dissolution ...  
4  public opinion in regard to the meaning of fal...  


# 1.3 Unique Texts

In [7]:
#1.3- List all the text sources listed in variable name

#Return only unique values in dataframe
unique_texts = texts.name.unique()

#Print unique values using for loop for formatting
print("Unique texts in data: \n")
for x in unique_texts: 
        print(x)

Unique texts in data: 

balbulus-early-life-charlemagne
beesly-queen-elizabeth
bible
carroll-alice-wonderland
chipman-earliest-electromagnetic-instruments
cia-world-factbook-1992
eckstein-quintus-claudius
fisher-quaker-colonies
gallienne-quest-of-golden-girl
gordon-quiet-talks-crowned-christ
hardy-madding-crowd
infiltrating-open-systems
kant-metaphysical-elements-ethics
karn-snowflakes
milton-paradise-lost
naval-academy-sound-military-decision
newsgroup
paper-compact-hash-tables
paper-data-compression
paper-logical-implementation-of-arithmetic
paper-programming-by-example
paper-search-for-autonomy
selected-polish-tales
shakespeare-as-you-like-it
unamuno-tragic-sense-of-life
vaneeden-quest
webster-early-european-history
why-speech-output
workshop-proceedings


# 1.3.1 Cleaning data

In [112]:
#Processes a given string
# * replaces all whitespace with a single splace
# * removes all non-English alphabets (numbers, special charachters, punctuation)
# * Removes stop words (e.g.- the, a, in, an, etc.)
def change_me(content):
    content = str(content)
    content = re.sub(r"[^A-Z ]+", "", content, 0, re.IGNORECASE)
    content = re.sub(r"[ ]{2,}", " ", content, 0, re.IGNORECASE)

    word_tokens = word_tokenize(content)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)

    filtered_sentence = [porter.stem(word) for word in filtered_sentence]
            
    return ' '.join(filtered_sentence)

#Do this for all the string "chunk" variables in texts
texts.chunk = texts.chunk.apply(change_me)

# 1.3.5 Splitting data intro training, test

In [114]:
#1.3.5- Split data into testing, training

#Samples the given dataframe for n random rows and returns training, testing, and validation data split on given number
def sampleTheDf(df, nbr, testSplit, valSplit):
    sTexts = df.dropna().sample(n = nbr, random_state = 1)
    
    #Break data into training, testing sets
    train, test = train_test_split(sTexts, train_size = testSplit, random_state = 1)
    train_t, val_t = train_test_split(train, train_size = valSplit, random_state = 1)

    #Reset index
    train = train.reset_index()
    test = test.reset_index()

    train_t = train_t.reset_index()
    val_t = val_t.reset_index()
    
    #To return all the data, putting them into a dictionary
    dataTable = {
      "train": train,
      "test": test,
      "train_t": train_t,
      "val_t": val_t
    }
    return dataTable 

# 1.4 Creating Dictionary of words (BOW)

In [115]:
#1.4- Create the Dictionary
def createDict(train, test):
    
    # merged training and validation data
    sentences = np.concatenate((train.chunk.values, test.chunk.values), axis=0)

    ## initialize the voctorizer
    vectorizer = CountVectorizer(min_df=0)

    ## create the dictionary
    # `fit` builds the vocabulary
    vectorizer.fit(sentences)

    ## transform your data into the BOW array
    # rows are sentences, columns are words
    X = vectorizer.transform(sentences).toarray()

    return X

# 1.5 Function to find cosine similarity

In [116]:
#1.5- Find cosine similarity between two vectors

#The cosine similarity = dot product / (length (v1) * length (v2))
#Returns number between 0 and 1 which tells us how similar the two vectors are
def cosine_sim(a, b):

    #Find the dot product# 1.4 Creating Dictionary of words (BOW)
    dot_ab = np.dot(a, b)
    
    #Find the length/magnite of vectors
    mag_a = np.linalg.norm(a)
    mag_b = np.linalg.norm(b)

    #Return cosine similarity
    return (dot_ab / (mag_a * mag_b))

#The dot product of 2 vectors is-
#a = 1, 2, 3, 4
#b = 1, 2, 3, 4
#a.b = 1*1 + 2*2 + 3*3 + 4*4

#The magnitude of a vector 
#a = 1, 2, 3, 4
#|a| = sqrt(12 + 22 + 32 + 42)

# 1.6, 1.7 Implement kNN

In [117]:
#1.6 Implementing KNN
def kNNSolver(k, X_train_mtx, Y_val, train_t):

    #m_score keeps track of cosine scores
    m_score = []
    
    #Find cosine for each value in training data
    for point in X_train_mtx:
        m_score.append(cosine_sim(point, Y_val))

    #Find top k values based on cosine score
    #Create df linking cosine scores to names
    a = pd.DataFrame.from_dict({'score': m_score, 'book': train_t.name})

    #Select the largest k elements
    a = a.nlargest(k, 'score')

    #Select the most common value from the subset of k elements
    a = a.groupby(['book']).agg(lambda x:x.value_counts().index[0])
    a = a.reset_index()
    a = a.book[0]

    return a

#returns dataframe of actual vs. predicted values
def kNN(k, X, train_t, val_t):
    
    #Create dataframe of actual vs. predicted names
    testCheck = val_t.name.reset_index()

    #Stores predicted values from KNN; added as col. to testCheck
    predictArray = []

    #Split X into train, test data
    
    #X_train_mtx = subset of matrix X to training data
    X_train_mtx = X[0:len(train_t)]
    
    #Y_val_mtx = subset of matrix X to validation or test data
    Y_val_mtx = X[len(train_t):len(train_t) + len(val_t)]

    #For each value in our testing data
    for Y_val in Y_val_mtx:
        val_P = kNNSolver(k, X_train_mtx, Y_val, train_t)
        predictArray.append(val_P)

    testCheck['predicted'] = predictArray
    return(testCheck[['name', 'predicted']])

# 1.8, 1.9 Testing implementation

In [118]:
#Due to memory and time limitations, only choosing 3000 samples
dataTable = sampleTheDf(texts, 3000, 0.9, 0.9) 

#Set up train, test based on return
train = dataTable.get('train')
test = dataTable.get('test')
train_t = dataTable.get('train_t')
val_t = dataTable.get('val_t')
X = createDict(train_t, val_t)

In [94]:
#Describing Data set sample
print("Training Data size:", len(train_t))
print("Testing Data size:", len(val_t))

timeArray = []
accArray = []

#Testing different versions of k
for k in [1, 5, 10, 20, 25]:
    start_time = time.time()
    c = kNN(k, X, train_t, val_t)
    c = len(c[c.name == c.predicted]) / len(c)
    timeTaken = time.time()-start_time

    timeArray.append(timeTaken)
    accArray.append(c)

    print("")
    print("k-value:", k)
    print("Accuracy:", c)
    print("Time taken:", timeTaken)

Training Data size: 2430
Testing Data size: 270

k-value: 1
Accuracy: 0.7925925925925926
Time taken: 91.97316336631775

k-value: 5
Accuracy: 0.6259259259259259
Time taken: 100.10655927658081

k-value: 10
Accuracy: 0.5259259259259259
Time taken: 94.31973338127136

k-value: 20
Accuracy: 0.42962962962962964
Time taken: 93.92343378067017

k-value: 25
Accuracy: 0.40370370370370373
Time taken: 98.62302422523499


In [96]:
print("Using kNN over our training and validation data suggests the highest accuracy is found with a k-value of 1 with an accuracy of 79.26%. This value also had the best performance, indicated by it taking the least time to compute.")
print("")
print("The following results are from using these findings over our training and testing data:")
print("")

#Train, Test already defined
X = createDict(train, test)
print("Training Data size:", len(train))
print("Testing Data size:", len(test))
print("")

#Testing data
start_time = time.time()
c = kNN(1, X, train, test)
c = len(c[c.name == c.predicted]) / len(c)
print("Time taken:", time.time()-start_time)
print("Accuracy:", c)


Using kNN over our training and validation data suggests the highest accuracy is found with a k-value of 1 with an accuracy of 79.26%. This value also had the best performance, indicated by it taking the least time to compute.

The following results are from using these findings over our training and testing data:

Training Data size: 2700
Testing Data size: 300

Time taken: 129.74847412109375
Accuracy: 0.7666666666666667


# 2.1 Create TF-IDF transformation

In [120]:
#Calculates TF, IDF, and returns TF-IDF matrix for data
def createTFIDFmatrix(X):
    X_TF = X/X.sum(axis=1)[:,None]
    X_IDF = np.log(X/X.sum(axis=0) + 1)
    X_TFIDF = np.multiply(X_TF,X_IDF)
    return X_TFIDF

# 2.2 Test TF-IDF transformation

In [121]:
X_TFIDF = createTFIDFmatrix(X)
print("Cosine similarity between same vector:", cosine_sim(X_TFIDF[1], X_TFIDF[1]))
print("")
print("Cosine similarity between diff vector w/o TF-IDF:", np.round_(cosine_sim(X[1], X[2]), decimals = 5))
print("Cosine similarity between diff vector w TF-IDF:", np.round_(cosine_sim(X_TFIDF[1], X_TFIDF[2]), decimals = 5))

Cosine similarity between same vector: 1.0

Cosine similarity between diff vector w/o TF-IDF: 0.02952
Cosine similarity between diff vector w TF-IDF: 2e-05


# 2.3 Testing TF-IDF on kNN

In [122]:
X = createDict(train_t, val_t)
X = createTFIDFmatrix(X)

#Describing Data set sample
print("Training Data size:", len(train_t))
print("Testing Data size:", len(val_t))

timeArray = []
accArray = []

#Testing different versions of k
for k in [1, 5, 10, 20, 25]:
    start_time = time.time()
    c = kNN(k, X, train_t, val_t)
    c = len(c[c.name == c.predicted]) / len(c)
    timeTaken = time.time()-start_time

    timeArray.append(timeTaken)
    accArray.append(c)

    print("")
    print("k-value:", k)
    print("Accuracy:", c)
    print("Time taken:", timeTaken)

Training Data size: 2430
Testing Data size: 270

k-value: 1
Accuracy: 0.6925925925925925
Time taken: 24.836672067642212

k-value: 5
Accuracy: 0.4962962962962963
Time taken: 26.776031017303467

k-value: 10
Accuracy: 0.34444444444444444
Time taken: 31.796534299850464

k-value: 20
Accuracy: 0.29259259259259257
Time taken: 30.60167169570923

k-value: 25
Accuracy: 0.27037037037037037
Time taken: 31.457271814346313


# 2.4 Interpreting Results

In [124]:
print("TF-IDF is MUCH faster, but less accurate than had the transformation not been applied. In general, as k increases, accuracy decreases. I think this may be the case because I've already removed all the stop words and cleaned the data quite a bit. Had those stop words still been in there, the results may have been different.")

TF-IDF is MUCH faster, but less accurate than had the transformation not been applied. In general, as k increases, accuracy decreases. I think this may be the case because I've already removed all the stop words and cleaned the data quite a bit. Had those stop words still been in there, the results may have been different.


# ========================================================

I forgot about the numbering/ question numbers, so we're pretending that didn't happen. ;) 

# 2.1 Inspecting Rotten Tomatoes dataset

In [151]:
reviews = pd.read_csv("D:\\OneDrive\\University of Washington Seattle\\Year 4 Quarter 2 (Winter)\\INFO 371\\Problem Set\\PS4\\reviews.csv", sep=',')

print('Shape of file:', reviews.shape)
print("")
print('Description of file: \n', reviews.dtypes.value_counts())
print("")
print("Head of file: \n", reviews.head())

Shape of file: (13442, 9)

Description of file: 
 object    7
int64     2
dtype: int64

Head of file: 
                critic  fresh    imdb  \
0         Derek Adams  fresh  114709   
1     Richard Corliss  fresh  114709   
2         David Ansen  fresh  114709   
3       Leonard Klady  fresh  114709   
4  Jonathan Rosenbaum  fresh  114709   

                                                link     publication  \
0  http://www.timeout.com/film/reviews/87745/toy-...        Time Out   
1  http://www.time.com/time/magazine/article/0,91...   TIME Magazine   
2                  http://www.newsweek.com/id/104199        Newsweek   
3  http://www.variety.com/review/VE1117941294.htm...         Variety   
4  http://onfilm.chicagoreader.com/movies/capsule...  Chicago Reader   

                                               quote          review_date  \
0  So ingenious in concept, design and execution ...  2009-10-04 00:00:00   
1                  The year's most inventive comedy.  2008-08-31 00:

# 2.2 Cleaning the data

In [152]:
#Removing cases where fresh isn't assigned and quotes are empty
reviews = reviews[(reviews.fresh != 'none') & (reviews.quote != '') & (reviews.quote != None)]

#Removing duplicates by grouping by all columns
reviews = pd.DataFrame(reviews.groupby(list(reviews)).size().reset_index())

#Cleaning data by removing stop words, punctuation, numbers, etc. from quotes
reviews.quote = reviews.quote.str.lower()
reviews.quote = reviews.quote.apply(change_me)

# 2.3, 2.4 Test over data

In [153]:
#Due to memory and time limitations, only choosing 3000 samples
dataTable = sampleTheDf(reviews, 3000, 0.9, 0.9) 

# 2.4.1 create dictionary and BOW of all quotes

In [155]:
#MODIFYING KNN TO BE USABLE OVER NEW DATA (chunk -> quote)
#1.4- Create the Dictionary
def createDict(train, test):
    
    # merged training and validation data
    sentences = np.concatenate((train.quote.values, test.quote.values), axis=0)

    ## initialize the voctorizer
    vectorizer = CountVectorizer(min_df=0)

    ## create the dictionary
    # `fit` builds the vocabulary
    vectorizer.fit(sentences)

    ## transform your data into the BOW array
    # rows are sentences, columns are words
    X = vectorizer.transform(sentences).toarray()

    return X

#Set up train, test based on return
train = dataTable.get('train')
test = dataTable.get('test')
train_t = dataTable.get('train_t')
val_t = dataTable.get('val_t')
X = createDict(train_t, val_t)

# 2.4.2 test fresh/rotten over different k-values; compute accuracy

In [165]:
#MODIFYING KNN TO BE USABLE OVER NEW DATA (name -> fresh) (chunk -> quote) (name -> actual)
#1.6 Implementing KNN
def kNNSolver(k, X_train_mtx, Y_val, train_t):

    #m_score keeps track of cosine scores
    m_score = []
    
    #Find cosine for each value in training data
    for point in X_train_mtx:
        m_score.append(cosine_sim(point, Y_val))

    #Find top k values based on cosine score
    #Create df linking cosine scores to names
    a = pd.DataFrame.from_dict({'score': m_score, 'fresh': train_t.fresh})

    #Select the largest k elements
    a = a.nlargest(k, 'score')

    #Select the most common value from the subset of k elements
    a = a.groupby(['fresh']).agg(lambda x:x.value_counts().index[0])
    a = a.reset_index()
    a = a.fresh[0]

    return a

#returns dataframe of actual vs. predicted values
def kNN(k, X, train_t, val_t):
    
    #Create dataframe of actual vs. predicted names
    testCheck = val_t.fresh.reset_index()

    #Stores predicted values from KNN; added as col. to testCheck
    predictArray = []

    #Split X into train, test data
    
    #X_train_mtx = subset of matrix X to training data
    X_train_mtx = X[0:len(train_t)]
    
    #Y_val_mtx = subset of matrix X to validation or test data
    Y_val_mtx = X[len(train_t):len(train_t) + len(val_t)]

    #For each value in our testing data
    for Y_val in Y_val_mtx:
        val_P = kNNSolver(k, X_train_mtx, Y_val, train_t)
        predictArray.append(val_P)

    testCheck['predicted'] = predictArray
    return(testCheck[['fresh', 'predicted']])

In [167]:
#Describing Data set sample
print("Training Data size:", len(train_t))
print("Testing Data size:", len(val_t))

timeArray = []
accArray = []

#Testing different versions of k
for k in [1, 5, 10, 20, 25]:
    start_time = time.time()
    c = kNN(k, X, train_t, val_t)
    c = len(c[c.fresh == c.predicted]) / len(c)
    timeTaken = time.time()-start_time

    timeArray.append(timeTaken)
    accArray.append(c)

    print("")
    print("k-value:", k)
    print("Accuracy:", c)
    print("Time taken:", timeTaken)

Training Data size: 2430
Testing Data size: 270

k-value: 1
Accuracy: 0.562962962962963
Time taken: 27.854827642440796

k-value: 5
Accuracy: 0.5407407407407407
Time taken: 30.965736627578735

k-value: 10
Accuracy: 0.5370370370370371
Time taken: 30.986567735671997

k-value: 20
Accuracy: 0.5370370370370371
Time taken: 31.330294370651245

k-value: 25
Accuracy: 0.5370370370370371
Time taken: 32.60551118850708


# 2.4.3 Transform your data into TF-IDF form and repeat k-NN

In [168]:
X = createDict(train_t, val_t)
X = createTFIDFmatrix(X)

#Describing Data set sample
print("Training Data size:", len(train_t))
print("Testing Data size:", len(val_t))

timeArray = []
accArray = []

#Testing different versions of k
for k in [1, 5, 10, 20, 25]:
    start_time = time.time()
    c = kNN(k, X, train_t, val_t)
    c = len(c[c.fresh == c.predicted]) / len(c)
    timeTaken = time.time()-start_time

    timeArray.append(timeTaken)
    accArray.append(c)

    print("")
    print("k-value:", k)
    print("Accuracy:", c)
    print("Time taken:", timeTaken)

Training Data size: 2430
Testing Data size: 270

k-value: 1
Accuracy: 0.6
Time taken: 10.979442834854126

k-value: 5
Accuracy: 0.5518518518518518
Time taken: 14.063071489334106

k-value: 10
Accuracy: 0.5370370370370371
Time taken: 14.14788556098938

k-value: 20
Accuracy: 0.5370370370370371
Time taken: 13.709917545318604

k-value: 25
Accuracy: 0.5370370370370371
Time taken: 13.917245149612427


# 2.4.4 Inspect, explain cases where tomato was correctly/incorrectly predicted

In [185]:
#MODIFYING KNN TO BE USABLE OVER NEW DATA (name -> fresh) (chunk -> quote) (name -> actual)
#1.6 Implementing KNN
def kNNSolver(k, X_train_mtx, Y_val, train_t):

    #m_score keeps track of cosine scores
    m_score = []
    
    #Find cosine for each value in training data
    for point in X_train_mtx:
        m_score.append(cosine_sim(point, Y_val))

    #Find top k values based on cosine score
    #Create df linking cosine scores to names
    a = pd.DataFrame.from_dict({'score': m_score, 'fresh': train_t.fresh})

    #Select the largest k elements
    a = a.nlargest(k, 'score')
    a = a.reset_index()

    a = a.groupby(['fresh']).agg(lambda x:x.value_counts().index[0])
    a = a.reset_index()

#    print(a.fresh[0])
#    print(a.score[0])
#    print(a)

    a = {
      "fresh": a.fresh[0],
      "score": a.score[0],
    }
    return(a)
    


#returns dataframe of actual vs. predicted values
def kNN(k, X, train_t, val_t):
    
    #Create dataframe of actual vs. predicted names
    testCheck = val_t[['fresh', 'quote']].reset_index()
    
    #Stores predicted values from KNN; added as col. to testCheck
    predictArray = []
    predictScore = []

    #Split X into train, test data
    
    #X_train_mtx = subset of matrix X to training data
    X_train_mtx = X[0:len(train_t)]
    
    #Y_val_mtx = subset of matrix X to validation or test data
    Y_val_mtx = X[len(train_t):len(train_t) + len(val_t)]

    #For each value in our testing data
    for Y_val in Y_val_mtx:
        val_P = kNNSolver(k, X_train_mtx, Y_val, train_t)
        predictArray.append(val_P.get('fresh'))
        predictScore.append(val_P.get('score'))
    testCheck['predicted'] = predictArray
    testCheck['score'] = predictScore
    return(testCheck)

X = createDict(train_t, val_t)
X = createTFIDFmatrix(X)

#Describing Data set sample
print("Training Data size:", len(train_t))
print("Testing Data size:", len(val_t))

timeArray = []
accArray = []

#Testing different versions of k
c = kNN(1, X, train_t, val_t)

Training Data size: 2430
Testing Data size: 270


In [202]:
cFalse = c[c.fresh != c.predicted]
pd.options.display.max_colwidth = 60
print(cFalse.sort_values(by=['score'], ascending=False)[['quote', 'fresh']].head())
#pd.options.display.max_colwidth = 50

                                                           quote   fresh
185                                                    worth see   fresh
107                                        troubl film need want   fresh
152                                                    fun watch   fresh
116  wisdom put bad experi behind us never rang true view mov...  rotten
215  mani dickey lumpi narr idea remain screenplay john boorm...  rotten


It's a bit hard to tell why the above quotes are false given they've been modified to the point where they're unrecognizable, but if I had to guess- it is probably because they used a negative word like "no" or "trouble" or had been sarcastic.

# 2.5 Summary

The pages worked better than the reviews with a higher accuraccy of 79% vs 60%. 

TF-IDF worked better than BOW (difference = 4% for k=1) for the review dataset, but 
BOW worked better than TF-IDF (difference = 9% for k=1) for the pages dataset.
Across both datasets, TF-IDF completed about 3 times faster than BOW.

All 4 cases had their highest accuracy at k=1, with it dropping as k increased.